# **VGG16 Model with 31 classes = 3100 pics of dataset of sneaker (Sneaker Recognition ) **

In [0]:
!pip uninstall tensorflow
!pip uninstall keras
!pip install tensorflow==1.13.0rc1
!pip install keras==2.1.3
#reinstall version

# import lib 

In [0]:
import keras
from math import ceil
import numpy as np
import pandas as pd
from keras.models import Sequential,Model,load_model,model_from_json
from keras.utils import to_categorical
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten,Input
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input

from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix

import matplotlib.pyplot as plt
import tensorflow

# open folder from google drive

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


# prepare data

In [7]:
#generate data form image
#preprocessing image by rescale form 0-255 to 0-1 for easy to use in CNN
path_clean = '/content/gdrive/My Drive/Project 62 - preaw+playfa/Clean'
path_test = '/content/gdrive/My Drive/Project 62 - preaw+playfa/test/test'
train_datagen = ImageDataGenerator(
                           rescale=1.0/255.0,
                           horizontal_flip=True,
                           shear_range=0.2,
                           zoom_range=0.2,
                           width_shift_range=0.1,
                           height_shift_range=0.1,
                           fill_mode='nearest')

test_datagen = ImageDataGenerator(
                           rescale=1.0/255.0,
                           horizontal_flip=True,
                           shear_range=0.2,
                           zoom_range=0.2,
                           width_shift_range=0.1,
                           height_shift_range=0.1,
                           fill_mode='nearest')

#ceil(dataset_size / batch_size)
batch_size=32
train_steps = ceil(2790/batch_size)
validation_steps = ceil(311/batch_size)


traindata = train_datagen.flow_from_directory(path_clean, target_size = (224, 224), batch_size = batch_size, class_mode='categorical')
validationdata = test_datagen.flow_from_directory(path_test, target_size = (224, 224), batch_size = batch_size, class_mode = 'categorical')


Found 2790 images belonging to 31 classes.
Found 311 images belonging to 31 classes.


# load pre-trained model vgg16

In [4]:
def pretrain_classifier_top ():
    model_vgg16 = VGG16(weights='imagenet', include_top=False)  ### no top layers , imagenet weights
    
    #freeze all layers
    for layer in model_vgg16.layers:
      layer.trainable = False
      
    #Set input format
    keras_input = Input(shape=(224,224,3), name = 'image_input')
    
    #new vgg16 model 
    output_vgg16_conv = model_vgg16(keras_input)
    
    #New classifier layers
    new = Flatten(name='flatten')(output_vgg16_conv)
    new = Dense(4096, activation='relu', name='fc1')(new)
    new = Dense(4096, activation='relu', name='fc2')(new)
    new = Dense(31, activation='softmax', name='predictions')(new)
    print("compiling")
    pretrained_model = Model(inputs=keras_input, outputs=new)
    #pretrained_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    pretrained_model.compile(loss='categorical_crossentropy',  optimizer=keras.optimizers.SGD(lr=0.001, momentum=0.8), metrics=['accuracy'])
    print("model compiled")
    
    return pretrained_model
model2 = pretrain_classifier_top() 

Instructions for updating:
Colocations handled automatically by placer.
58892288/58889256 [==============================] - 2s 0us/step
compiling
model compiled


# train model 

In [ ]:
showmodel = model2.fit_generator(traindata,
                                   steps_per_epoch=train_steps,
                                   epochs=20,
                                  validation_data=validationdata,
                                   validation_steps=validation_steps)
print("----------------------------------------------------------------------------------------------------------")


# show accuracy and loss

In [9]:
#eveluate Model
train_loss, train_acc = model2.evaluate_generator(generator=traindata,steps=train_steps)
print('\nTrain accuracy:', train_acc,train_loss)


test_loss, test_acc = model2.evaluate_generator(generator=validationdata,steps=validation_steps)
print('\nTest accuracy:', test_acc,test_loss)


Train accuracy: 0.8831541219065266 0.6810183269148659

Test accuracy: 0.8392282952449712 0.7733322153904048


In [0]:
#json_file = open('model_architectureReal.json', 'r')
#loaded_model_json = json_file.read()
#json_file.close()
#loaded_model = model_from_json(loaded_model_json)
# load weights into new model
#loaded_model.load_weights("modelReal.h5")

# prediction part

In [10]:
#create class 
test_datagen = ImageDataGenerator(rescale=1.0/255.0)
testdata = test_datagen.flow_from_directory(path_test, target_size = (224, 224),batch_size=32,shuffle=False)
test_steps_per_epoch = ceil(testdata.samples / testdata.batch_size)

predict = model2.predict_generator(testdata,steps=test_steps_per_epoch,verbose=1)
predicted_class=np.argmax(predict,axis=1)

Found 311 images belonging to 31 classes.
10/10 [==============================] - 290s 29s/step


In [11]:
#predict data form 100 image test set
true_class = testdata.classes
class_labels = list(testdata.class_indices.keys())

report = metrics.classification_report(true_class, predicted_class, target_names=class_labels)
#print(report)    

#print(true_class)
for i in range (0,311):
  print(i,true_class[i],' ',predicted_class[i])  

0 0   0
1 0   0
2 0   0
3 0   0
4 0   0
5 0   0
6 0   0
7 0   0
8 0   0
9 0   0
10 1   1
11 1   1
12 1   1
13 1   1
14 1   6
15 1   1
16 1   1
17 1   1
18 1   1
19 1   1
20 2   2
21 2   2
22 2   2
23 2   2
24 2   2
25 2   2
26 2   13
27 2   2
28 2   2
29 2   2
30 3   3
31 3   2
32 3   3
33 3   3
34 3   16
35 3   3
36 3   7
37 3   3
38 3   3
39 3   3
40 4   4
41 4   4
42 4   24
43 4   7
44 4   4
45 4   3
46 4   4
47 4   29
48 4   4
49 4   22
50 5   5
51 5   7
52 5   5
53 5   5
54 5   5
55 5   5
56 5   5
57 5   5
58 5   5
59 5   5
60 6   6
61 6   6
62 6   6
63 6   6
64 6   6
65 6   6
66 6   6
67 6   6
68 6   6
69 6   10
70 7   7
71 7   7
72 7   7
73 7   7
74 7   7
75 7   7
76 7   7
77 7   7
78 7   7
79 7   7
80 8   8
81 8   8
82 8   8
83 8   8
84 8   8
85 8   8
86 8   8
87 8   8
88 8   8
89 8   8
90 9   9
91 9   9
92 9   9
93 9   9
94 9   9
95 9   9
96 9   9
97 9   9
98 9   9
99 9   9
100 10   10
101 10   10
102 10   10
103 10   10
104 10   10
105 10   10
106 10   10
107 10   10
108 10  

# save model , weight and prediction into .h5 , .json , .csv file

In [0]:
#save model & weight & result from 100 testing data into .csv file
#filenames=testdata.filenames
filenames=true_class
print ("",len(filenames))
print ("",len(predicted_class))
results=pd.DataFrame({"Filename":filenames,
                     "Predictions":predicted_class})
results.to_csv('Result_16042020.csv',index=False)
with open('Result_16042020.json', 'w') as f:
    f.write(model2.to_json())
model2.save_weights('Result_16042020_1.h5')

from google.colab import files
#files.download('Final31classesResult.csv')
model2.save('Result_16042020_2.h5') 
files.download('Result_16042020_1.h5')
files.download('Result_16042020_2.h5')
files.download('Result_16042020.json')
files.download('Result_16042020t.csv')

# show graph plot from train and test set

In [0]:
#กราฟแสดงค่าaccuracy
plt.plot(showmodel.history['acc'])
plt.plot(showmodel.history['val_acc'])
plt.title('model Accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
#กราฟแสดงค่าloss
plt.plot(showmodel.history['loss'])
plt.plot(showmodel.history['val_loss'])
plt.title('model Loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()



In [0]:
print(model2.summary()) # show model summary